In [23]:
import torch
from __future__ import annotations
from torchdrug import core, tasks, datasets
import yaml
import protbert as pb
import lmdb
import torch.optim as optim
from torch.utils import data as torch_data
import os
from torchdrug.datasets import Solubility
from torchdrug import data, utils


Datos

In [24]:
#archivo de configuración
def load_config(cfg_file):
    with open(cfg_file, "r") as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    return config

config_path = "/Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11/solubility_ProtBert.yaml"
config = load_config(config_path)

In [ ]:
ds.data[0].residue_type

In [25]:
ds=Solubility("/Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11",lazy=True)

11:19:31   Extracting /Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11/solubility.tar.gz to /Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11


Constructing proteins from sequences: 100%|██████████| 71419/71419 [00:00<00:00, 286039.90it/s]


In [153]:
len(ds.data[0].id2residue_symbol)

20

In [149]:
ds.data[0].residue_type

tensor([ 0, 14,  7,  8, 12,  5,  9,  8, 16,  0, 15,  5, 18, 11, 16, 12,  2,  7,
         5, 10,  4,  8,  1, 12,  1,  9, 13, 13, 12,  2,  0, 10, 17,  8,  1,  0,
         4,  1,  1, 13,  2,  1, 13, 13, 17,  4,  1,  1, 12,  4,  4,  8,  2, 12,
        14,  5,  8,  0, 10,  8, 17,  9,  9,  3,  4,  4,  3, 18, 13,  5, 10, 13,
         4,  5, 17,  7,  7, 11, 10, 11,  4,  9, 10, 17,  7, 15, 10,  2,  7, 12,
         9, 19,  5,  4, 13, 13,  8, 17, 13, 19,  7,  8, 10, 15, 12,  5,  5, 10,
         1, 10,  7, 12, 17,  4,  1, 17,  0,  8,  5,  2, 13,  7,  7,  1,  1,  4,
         5, 12,  8, 14,  2,  9,  8, 10,  8,  7, 18,  0,  1, 12, 12,  7, 17,  4,
         7,  1, 15,  1,  9,  5,  5,  7,  0,  8,  3,  0,  5, 16,  2,  1, 17,  8,
        11,  3,  9, 15,  3,  5, 10, 10,  3, 10,  0,  7,  8,  1,  2,  8, 14, 13,
         0,  8,  5, 18,  0,  7,  0, 10,  1,  4,  7,  0,  8,  9,  3,  4, 10, 10,
         2,  5, 10,  2,  4,  4, 17,  8,  8,  9, 12, 16, 13, 13, 16, 17,  2, 12,
        19, 10,  4,  3,  5, 11,  5,  6, 

In [150]:
ds.sequences[0]

'GMILKTNLFGHTYQFKSITDVLAKANEEKSGDRLAGVAAESAEERVAAKVVLSKMTLGDLRNNPVVPYETDEVTRIIQDQVNDRIHDSIKNWTVEELREWILDHKTTDADIKRVARGLTSEIIAAVTKLMSNLDLIYGAKKIRVIAHANTTIGLPGTFSARLQPNHPTDDPDGILASLMEGLTYGIGDAVIGLNPVDDSTDSVVRLLNKFEEFRSKWDVPTQTCVLAHVKTQMEAMRRGAPTGLVFQSIAGSEKGNTAFGFDGATIEEARQLALQSGAATGPNVMYFETGQGSELSSDAHFGVDQVTMEARCYGFAKKFDPFLVNTVVGFIGPEYLYDSKQVIRAGLEDHFMGKLTGISMGCDVCYTNHMKADQNDVENLSVLLTAAGCNFIMGIPHGDDVMLNYQTTGYHETATLRELFGLKPIKEFDQWMEKMGFSENGKLTSRAGDASIFLK'

In [156]:
ds.data[0].id2residue_symbol.values()

dict_values(['G', 'A', 'S', 'P', 'V', 'T', 'C', 'I', 'L', 'N', 'D', 'Q', 'K', 'E', 'M', 'H', 'F', 'R', 'Y', 'W'])

In [ ]:
'L',
    'A','G','V','E','S','I','K','R','D','T','P','N',
    'Q','F','Y','M','H','C','W','X','U','B','Z','O'

In [ ]:
i =ds.data[0].residue_type
def construct_mapping(tokenizer,):
    mapping = [0] * len(data.Protein.id2residue_symbol)
    for i, token in data.Protein.id2residue_symbol.items():
        mapping[i] = tokenizer._convert_token_to_id(token)
    mapping = torch.tensor(mapping)
    return mapping

i = mapping[i]

In [84]:
len(ds.sequences[0])

455

In [34]:
type(ds.sequences)

torchdrug.datasets.solubility.Solubility

In [32]:
type(ds.targets['solubility'])

list

In [33]:
sequences=ds.sequences
targets=ds.targets['solubility']

In [95]:
sequences[1]

'MAHHHHHHMSFFRMKRRLNFVVKRGIEELWENSFLDNNVDMKKIEYSKTGDAWPCVLLRKKSFEDLHKLYYICLKEKNKLLGEQYFHLQNSTKMLQHGRLKKVKLTMKRILTVLSRRAIHDQCLRAKDMLKKQEEREFYEIQKFKLNEQLLCLKHKMNILKKYNSFSLEQISLTFSIKKIENKIQQIDIILNPLRKETMYLLIPHFKYQRKYSDLPGFISWKKQNIIALRNNMSKLHRLY'

In [110]:
aminoacidos = [
    'A','R','N','D','C','Q','E','G','H',
    'I','L','K','M','F','P','S','T','W','Y','V'
]
#diccionario de mapeo
letter_to_number = {letter: index + 1 for index, letter in enumerate(aminoacidos)}
# Función para convertir secuencias de letras a secuencias de números
def convert_sequences_to_numbers(sequences, mapping):
    return [[mapping[letter] for letter in seq] for seq in sequences]

sequences_number=convert_sequences_to_numbers(sequences, letter_to_number)

In [116]:
sequences_number[1]

[13,
 1,
 9,
 9,
 9,
 9,
 9,
 9,
 13,
 16,
 14,
 14,
 2,
 13,
 12,
 2,
 2,
 11,
 3,
 14,
 20,
 20,
 12,
 2,
 8,
 10,
 7,
 7,
 11,
 18,
 7,
 3,
 16,
 14,
 11,
 4,
 3,
 3,
 20,
 4,
 13,
 12,
 12,
 10,
 7,
 19,
 16,
 12,
 17,
 8,
 4,
 1,
 18,
 15,
 5,
 20,
 11,
 11,
 2,
 12,
 12,
 16,
 14,
 7,
 4,
 11,
 9,
 12,
 11,
 19,
 19,
 10,
 5,
 11,
 12,
 7,
 12,
 3,
 12,
 11,
 11,
 8,
 7,
 6,
 19,
 14,
 9,
 11,
 6,
 3,
 16,
 17,
 12,
 13,
 11,
 6,
 9,
 8,
 2,
 11,
 12,
 12,
 20,
 12,
 11,
 17,
 13,
 12,
 2,
 10,
 11,
 17,
 20,
 11,
 16,
 2,
 2,
 1,
 10,
 9,
 4,
 6,
 5,
 11,
 2,
 1,
 12,
 4,
 13,
 11,
 12,
 12,
 6,
 7,
 7,
 2,
 7,
 14,
 19,
 7,
 10,
 6,
 12,
 14,
 12,
 11,
 3,
 7,
 6,
 11,
 11,
 5,
 11,
 12,
 9,
 12,
 13,
 3,
 10,
 11,
 12,
 12,
 19,
 3,
 16,
 14,
 16,
 11,
 7,
 6,
 10,
 16,
 11,
 17,
 14,
 16,
 10,
 12,
 12,
 10,
 7,
 3,
 12,
 10,
 6,
 6,
 10,
 4,
 10,
 10,
 11,
 3,
 15,
 11,
 2,
 12,
 7,
 17,
 13,
 19,
 11,
 11,
 10,
 15,
 9,
 14,
 12,
 19,
 6,
 2,
 12,
 19,
 16,
 4,
 11,
 15,
 8

In [138]:
#longitud del string más larga
max_length = min(len(s) for s in sequences)

print("La longitud del string más largo es:", max_length)

La longitud del string más largo es: 19


In [66]:
targets[0]

1

In [42]:
ds.num_samples #train,valid,test

[62478, 6942, 1999]

#separamos como la función split (en orden)
```python
def split(self):
    offset = 0
    splits = []
    for num_sample in self.num_samples:
        split = torch_data.Subset(self, range(offset, offset + num_sample))
        splits.append(split)
        offset += num_sample
    return splits
```

In [63]:
train_sequences=sequences[0:ds.num_samples[0]]
valid_sequences=sequences[ds.num_samples[0]:ds.num_samples[0]+ds.num_samples[1]]
test_sequences=sequences[ds.num_samples[0]+ds.num_samples[1]:]

train_target=targets[0:ds.num_samples[0]]
valid_target=targets[ds.num_samples[0]:ds.num_samples[0]+ds.num_samples[1]]
test_target=targets[ds.num_samples[0]+ds.num_samples[1]:]


In [64]:
print("numero de secuencias por set:\n")
print("train: ",len(train_sequences))
print("valid: ",len(valid_sequences))
print("test: ",len(test_sequences))

numero de secuencias por set:

train:  62478
valid:  6942
test:  1999


In [65]:
print("numero de targets por set:\n")
print("train: ",len(train_target))
print("valid: ",len(valid_target))
print("test: ",len(test_target))

numero de targets por set:

train:  62478
valid:  6942
test:  1999


In [74]:
type(train_sequences)

list

In [70]:
def read_lmdb(lmdb_path, max_items=10):
    env = lmdb.open(lmdb_path, readonly=True)
    with env.begin() as txn:
        cursor = txn.cursor()
        count = 0
        for key, value in cursor:
            print(f"Key: {key}")
            print(f"Value: {value[:3000]}...")  # Trunca el valor para que sea legible
            print("------")
            count += 1
            if count >= max_items:
                break

# Ruta a uno de los archivos LMDB
lmdb_path = os.path.expanduser("/Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11/solubility/solubility_train.lmdb")

# Leer y visualizar contenido del archivo LMDB
print(f"Reading {lmdb_path}")
read_lmdb(lmdb_path, max_items=10)

Reading /Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11/solubility/solubility_train.lmdb
Key: b'0'
Value: b'\x80\x03}q\x00(X\x07\x00\x00\x00primaryq\x01X\xc7\x01\x00\x00GMILKTNLFGHTYQFKSITDVLAKANEEKSGDRLAGVAAESAEERVAAKVVLSKMTLGDLRNNPVVPYETDEVTRIIQDQVNDRIHDSIKNWTVEELREWILDHKTTDADIKRVARGLTSEIIAAVTKLMSNLDLIYGAKKIRVIAHANTTIGLPGTFSARLQPNHPTDDPDGILASLMEGLTYGIGDAVIGLNPVDDSTDSVVRLLNKFEEFRSKWDVPTQTCVLAHVKTQMEAMRRGAPTGLVFQSIAGSEKGNTAFGFDGATIEEARQLALQSGAATGPNVMYFETGQGSELSSDAHFGVDQVTMEARCYGFAKKFDPFLVNTVVGFIGPEYLYDSKQVIRAGLEDHFMGKLTGISMGCDVCYTNHMKADQNDVENLSVLLTAAGCNFIMGIPHGDDVMLNYQTTGYHETATLRELFGLKPIKEFDQWMEKMGFSENGKLTSRAGDASIFLKq\x02X\n\x00\x00\x00solubilityq\x03K\x01u.'...
------
Key: b'1'
Value: b'\x80\x03}q\x00(X\x07\x00\x00\x00primaryq\x01X\xf0\x00\x00\x00MAHHHHHHMSFFRMKRRLNFVVKRGIEELWENSFLDNNVDMKKIEYSKTGDAWPCVLLRKKSFEDLHKLYYICLKEKNKLLGEQYFHLQNSTKMLQHGRLKKVKLTMKRILTVLSRRAIHDQCLRAKDMLKKQEEREFYEIQKFKLNEQLLCLKHKMNILKKYNSFSLEQISLTFSIKKIENKIQQIDIILNPLRKETMYLL

In [120]:
train_sequences[:10][:]

['GMILKTNLFGHTYQFKSITDVLAKANEEKSGDRLAGVAAESAEERVAAKVVLSKMTLGDLRNNPVVPYETDEVTRIIQDQVNDRIHDSIKNWTVEELREWILDHKTTDADIKRVARGLTSEIIAAVTKLMSNLDLIYGAKKIRVIAHANTTIGLPGTFSARLQPNHPTDDPDGILASLMEGLTYGIGDAVIGLNPVDDSTDSVVRLLNKFEEFRSKWDVPTQTCVLAHVKTQMEAMRRGAPTGLVFQSIAGSEKGNTAFGFDGATIEEARQLALQSGAATGPNVMYFETGQGSELSSDAHFGVDQVTMEARCYGFAKKFDPFLVNTVVGFIGPEYLYDSKQVIRAGLEDHFMGKLTGISMGCDVCYTNHMKADQNDVENLSVLLTAAGCNFIMGIPHGDDVMLNYQTTGYHETATLRELFGLKPIKEFDQWMEKMGFSENGKLTSRAGDASIFLK',
 'MAHHHHHHMSFFRMKRRLNFVVKRGIEELWENSFLDNNVDMKKIEYSKTGDAWPCVLLRKKSFEDLHKLYYICLKEKNKLLGEQYFHLQNSTKMLQHGRLKKVKLTMKRILTVLSRRAIHDQCLRAKDMLKKQEEREFYEIQKFKLNEQLLCLKHKMNILKKYNSFSLEQISLTFSIKKIENKIQQIDIILNPLRKETMYLLIPHFKYQRKYSDLPGFISWKKQNIIALRNNMSKLHRLY',
 'MGSDKIHHHHHHMEKSIQDTIHGVIKLEDWMVEIVDTPQFQRLRRINQLGFANLVYPGANHTRFEHSLGVMHVTRILQERMGFDDVVVAAALLHDVGHAPFSHGSERLLEKYASYNHETISKVVRGELKDVLKNLGFRISEIEAIVTGKRRSVVNGEIDADRMDYLVRDSHYTGVAYGVFDIYRLIDKIKFDGAVVIEQGGVKAAESLLISRFLMYPTVYFHHVCRIARKMYERAMERIIEAGFEAENLLEMDDVDAMMLLKEKERETYDRLNNRRLFKRAIYVSRRSLDFRE

In [117]:
type(sequences_number)

list

In [126]:
config['batch_size']=32
from torch.utils.data import DataLoader
import numpy as np
class PeptideBERTDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

        self.length = len(self.input_ids)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        attention_mask = self.attention_masks[idx]
        label = self.labels[idx]

        return {
            'input_ids': torch.tensor(input_id, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.float)
        }

def atention_mask(array_sequences, max_length):
    m = len(array_sequences)
    atention_mask_sequence = np.zeros((m, max_length), dtype=np.float64)

    for i, seq in enumerate(array_sequences):
        seq_len = min(len(seq), max_length)
        atention_mask_sequence[i, :seq_len] = 1

    return atention_mask_sequence


def load_data(sequences, targets, ds, max_length):
    print(f'{"="*30}{"DATA":^20}{"="*30}')
    
    # Convertir secuencias a números
    #sequences = convert_sequences_to_numbers(sequences, letter_to_number)
    
    # Inicializar arrays para las secuencias rellenadas y las máscaras de atención
    train_sequences = np.zeros((ds.num_samples[0], max_length), dtype=np.float64)
    valid_sequences = np.zeros((ds.num_samples[1], max_length), dtype=np.float64)
    test_sequences = np.zeros((len(sequences) - ds.num_samples[0] - ds.num_samples[1], max_length), dtype=np.float64)
    
    # Rellenar las secuencias
    for i, seq in enumerate(sequences[:ds.num_samples[0]]):
        train_sequences[i, :len(seq)] = seq
    for i, seq in enumerate(sequences[ds.num_samples[0]:ds.num_samples[0] + ds.num_samples[1]]):
        valid_sequences[i, :len(seq)] = seq
    for i, seq in enumerate(sequences[ds.num_samples[0] + ds.num_samples[1]:]):
        test_sequences[i, :len(seq)] = seq
    
    # Crear las máscaras de atención
    attention_mask_train = (train_sequences > 0).astype(np.float64)
    attention_mask_val = (valid_sequences > 0).astype(np.float64)
    attention_mask_test = (test_sequences > 0).astype(np.float64)
    
    # Convertir los targets a arrays
    train_target = np.array(targets[:ds.num_samples[0]])
    valid_target = np.array(targets[ds.num_samples[0]:ds.num_samples[0] + ds.num_samples[1]])
    test_target = np.array(targets[ds.num_samples[0] + ds.num_samples[1]:])
    train_dataset = PeptideBERTDataset(input_ids=train_sequences, attention_masks=attention_mask_train, labels=train_target)
    val_dataset = PeptideBERTDataset(input_ids=valid_sequences, attention_masks=attention_mask_val, labels=valid_target)
    test_dataset = PeptideBERTDataset(input_ids=test_sequences, attention_masks=attention_mask_test, labels=test_target)

    train_data_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True
    )

    val_data_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=config['batch_size'],
        shuffle=False
    )

    print('Batch size: ', config['batch_size'])

    print('Train dataset samples: ', len(train_dataset))
    print('Validation dataset samples: ', len(val_dataset))
    print('Test dataset samples: ', len(test_dataset))

    print('Train dataset batches: ', len(train_data_loader))
    print('Validation dataset batches: ', len(val_data_loader))
    print('Test dataset batches: ', len(test_data_loader))

    print()

    return train_data_loader, val_data_loader, test_data_loader

In [127]:
train_data_loader, val_data_loader, test_data_loader = load_data(sequences_number,targets,ds,max_length)

==============================        DATA        ==============================
Batch size:  32
Train dataset samples:  62478
Validation dataset samples:  6942
Test dataset samples:  1999
Train dataset batches:  1953
Validation dataset batches:  217
Test dataset batches:  63



In [130]:
train_data_loader.dataset[7410]['input_ids'].size()

torch.Size([1200])

PeptideBert

In [ ]:
# Función para cargar el modelo guardado
def load_trained_model(save_path, config):
    model = create_model(config)
    checkpoint = torch.load(save_path, map_location=config['device'])
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

ProtBert

In [71]:
from transformers import BertModel
#Clase solubilidad, que hereda de torch.nn.Module 
class solubilidad_bert(torch.nn.Module):
    def __init__(self, bert_config): #constructor de la clase con parametros para bert
        #llamamos al constructor de la clase base torch.nn.Module.
        super(solubilidad_bert, self).__init__()

        #cargamos el modelo preentrenado
        self.protbert = BertModel.from_pretrained(
            'Rostlab/prot_bert_bfd', 
            config=bert_config, #pasamos la onfiguracion del modelo
            ignore_mismatched_sizes=True 
        ) 
        #clasificacion
        self.head = torch.nn.Sequential( 
            torch.nn.Linear(bert_config.hidden_size, 1), #toma la salida de protVert y la convierte en un valor
            torch.nn.Sigmoid()
        )
    #definimos el metodo Fordward que especifica como procesar los datos
    def forward(self, inputs, attention_mask):
        #pasamos las entradas a través de ProtBert
        output = self.protbert(inputs, attention_mask=attention_mask)
        #usamos la salida de ProtBert como entrada a la capa de clasificacion
        return self.head(output.pooler_output)
 

In [27]:
# Configurar el modelo ProtBert
model_config = config['task']['model']
model = pb.ProtBert(model_config['path'], readout="pooler")

11:28:43   Extracting /Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11/protbert-model-weights/prot_bert_bfd.zip to /Users/ely/Documents/Maestria/proyecto_tecnologico/cimat2024-proyecto/Tareas/2024_07_11/protbert-model-weights


In [28]:
task = tasks.PropertyPrediction(model, task=ds.tasks,
                                criterion="ce", metric=("acc", "mcc"))

In [29]:
# Configurar el optimizador
optimizer_config = config['optimizer']
optimizer = optim.Adam(task.parameters(), lr=optimizer_config['lr'])

In [30]:
# Configurar el motor de entrenamiento
engine_config = config['engine']
if not "scheduler" in engine_config:
        scheduler = None
else:
        engine_config.scheduler.optimizer = optimizer
        scheduler = core.Configurable.load_config_dict(engine_config.scheduler)

In [31]:
fix_encoder=config['fix_encoder']
if fix_encoder:
    for p in task.model.parameters():
        p.requires_grad = False

In [56]:
ds.lazy=False

In [58]:
train_set.lazy=False
test_set.lazy=False
valid_set.lazy=False

In [59]:
solver = core.Engine(task, train_set=train_set,valid_set= valid_set, test_set=test_set , optimizer=optimizer, scheduler=scheduler, batch_size=1024,gradient_interval=None)

12:12:44   Preprocess training set


AttributeError: 'Solubility' object has no attribute 'smiles_list'

In [43]:
train_set[1000]

{'graph': None, 'solubility': 1}

In [33]:
lazy=True

In [34]:
# Entrenar el modelo
train_config = config['train']
solver.train(num_epoch=train_config['num_epoch'])

11:28:52   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
11:28:52   Epoch 0 begin


TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [61]:
from collections.abc import Mapping

In [62]:
Mapping

collections.abc.Mapping

In [ ]:
def load_data(config):
    print(f'{"="*30}{"DATA":^20}{"="*30}')
    #editamos para tarea
    with np.load(f'./data/{config["task"]}/train.npz') as train,\
         np.load(f'./data/{config["task"]}/val.npz') as val,\
         np.load(f'./data/{config["task"]}/test.npz') as test:
        train_inputs = train['inputs']
        train_labels = train['labels']
        val_inputs = val['inputs']
        val_labels = val['labels']
        test_inputs = test['inputs']
        test_labels = test['labels']

    attention_mask = np.asarray(train_inputs > 0, dtype=np.float64)
    attention_mask_val = np.asarray(val_inputs > 0, dtype=np.float64)
    attention_mask_test = np.asarray(test_inputs > 0, dtype=np.float64)

    train_dataset = PeptideBERTDataset(input_ids=train_inputs, attention_masks=attention_mask, labels=train_labels)
    val_dataset = PeptideBERTDataset(input_ids=val_inputs, attention_masks=attention_mask_val, labels=val_labels)
    test_dataset = PeptideBERTDataset(input_ids=test_inputs, attention_masks=attention_mask_test, labels=test_labels)


In [131]:
from transformers import BertModel, BertConfig, logging

In [134]:
BertConfig["max_position_embeddings"]

TypeError: 'type' object is not subscriptable